In [1]:
from datetime import date
from tqdm import tqdm
import shutil
import os

from thumbnails import ThumbnailDef, Thumbnailer, ThumbnailParams
from db_connect import DbConnector
from photo_export import PhotoExporter
from path_management import PathManager
from bucket_connect import BucketConnector

In [2]:
from_date = date(2024, 4, 13)
to_date = date(2024, 4, 13)

In [3]:
small = ThumbnailDef(120, 'small')
medium = ThumbnailDef(400, 'medium')
large = ThumbnailDef(1600, 'large')

sizes = [small, large, medium] 
thumbnailer = Thumbnailer(sizes)

In [4]:
paths = PathManager(thumbnailer.name_modifiers)
test_connector = DbConnector('./test.postgres_config', paths)
prod_connector = DbConnector('./prod.postgres_config', paths)
exporter = PhotoExporter(paths)

games = prod_connector.get_games(from_date, to_date)

In [6]:
for game in tqdm(games):
    exporter.export_photos_for_game(game)

100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


At this point, review the photos in the "preview" directories and remove any that shouldn't be uploaded. Removing it from the preview directory is sufficient.

In [7]:
to_thumbnail: list[ThumbnailParams] = []
for game in games:
    to_thumbnail.extend(exporter.get_photos_to_thumbnail(game))

In [8]:
thumbnailer.thumbnail_many(to_thumbnail)

In [9]:
bucket = BucketConnector('./prod.bucket_config', paths)

In [10]:
for game in games:
    to_upload = exporter.get_photos_to_upload(game)
    print(f'Uploading/Validating {len(to_upload)} photos for {game.Name}')
    for photo in to_upload:
        root_path = paths.temp_dir(game, photo)
        bucket.upload_all_files(root_path, photo)
    prod_connector.import_resources(game, to_upload)

Uploading/Validating 57 photos for 4/13/24 Los Angeles Angels at Boston Red Sox


In [11]:
for game in games:
    root_path = paths.base_dir(game)
    if os.path.exists(root_path):
        shutil.rmtree(root_path)